In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import json
import malaya
from tqdm import tqdm
from unidecode import unidecode
import random

/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3386
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3904
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [4]:
import re

def cleaning(string):
    splitted = malaya.text.function.split_into_sentences(string)
    if not len(splitted):
        splitted = '. '.join([k.strip() for k in string.split('.') if len(k.strip())])
    if splitted[0][0] == '-':
        splitted[0] = splitted[0].replace('- ','')
    points = [f'{no + 1}. {s}' for no, s in enumerate(splitted)]
    points = ' '.join(points)
    return points

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from malaya.text.vectorizer import SkipGramCountVectorizer, SkipGramTfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation

stopwords = malaya.text.function.get_stopwords()

In [6]:
vectorizer = SkipGramCountVectorizer(
    max_df = 0.95,
    min_df = 1,
    ngram_range = (1, 3),
    stop_words = stopwords,
    skip = 2
)
svd = TruncatedSVD(n_components = 30)
model = malaya.summarization.extractive.sklearn(svd, vectorizer)

In [7]:
vectorizer = SkipGramTfidfVectorizer(
    max_df = 0.95,
    min_df = 1,
    ngram_range = (1, 3),
    stop_words = stopwords,
    skip = 2
)
svd = TruncatedSVD(n_components = 30)
model_tfidf = malaya.summarization.extractive.sklearn(svd, vectorizer)

In [8]:
months = {
    'january',
    'jan',
    'januari',
    'february',
    'feb',
    'februari',
    'march',
    'mac',
    'april',
    'apr',
    'may',
    'mei',
    'june',
    'jun',
    'july',
    'julai',
    'august',
    'ogos',
    'aug',
    'september',
    'sep',
    'october',
    'oktober',
    'oct',
    'november',
    'nov',
    'december',
    'disember',
    'dec',
    'utusan',
    'malaysiakini',
    'astroawani',
    'bernama',
    'com',
}

In [9]:
def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

In [17]:
from glob import glob
from bs4 import BeautifulSoup

files = sorted(glob('*.nested.semisupervised'))
files

['berita-bisnes.json.nested.semisupervised',
 'berita-dunia.json.nested.semisupervised',
 'berita-hiburan.json.nested.semisupervised',
 'berita-malaysia.json.nested.semisupervised',
 'berita-politik.json.nested.semisupervised',
 'berita-sukan.json.nested.semisupervised',
 'berita-teknologi.json.nested.semisupervised',
 'gaya-hidup.json.nested.semisupervised']

In [15]:
with open(files[0]) as fopen:
    for l in fopen:
        data = json.loads(l)
        break

In [26]:
before, after = [], []

count = 0
rejected = []
languages = []
accepted = []
para = []
malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama'
}

def reject(data):
    if data['news'] in malaysian_news:
        return False
    if any([n in data['top-image'] for n in malaysian_news]):
        return False
    if any([n in data['url'] for n in malaysian_news]):
        return False
    if 'com.my' in data['top-image']:
        return False
    if data['language'] == 'malay':
        return False
    if 'Siaran Pers' in data['news']:
        return True
    if '.id' in data['news']:
        return True
    
    return True

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            soup = BeautifulSoup(data['r']['response']['articleBody'], "lxml")
            text = BeautifulSoup(soup.text, 'lxml').text
            text = re.sub(r'[ ]+', ' ', text).strip()
            if 'kindly register' in text.lower() or 'disabled in your browser' in text.lower():
                continue
            if len(text.split()) < 30:
                continue

            # accepted.append(data)
            summaries = [malaya.text.function.remove_empty_parenthesis(s) for s in data['semisupervised-summaries']]
            for s in summaries:
                if len(s.split()) > 20:
                    accepted.append((s, text))

            keywords_rake = malaya.keyword.extractive.rake(text, 
                                                      top_k = random.randint(25, 50))
            keywords_rake = [simple_cleaning(k[1]) for k in keywords_rake if len(k[1].split()) >= 3 and len(k[1]) > 20 \
                            and len(set(k[1].lower().replace('-', '').split()) & months) == 0]

            already = set()
            filtered = []
            for k in keywords_rake:
                if k.lower() in already:
                    continue
                else:
                    already.add(k.lower())
                    filtered.append(k)

            keywords_rake = filtered

            if len(keywords_rake) > 5:
                # print(keywords_rake)
                accepted.append(('. '.join(keywords_rake), text))

    #         if random.random() > 0.1:
    #             continue

    #         try:
    #             extractive = model_tfidf.sentence_level(data['text'], top_k = random.randint(3, 5))['summary']
    #             splitted = malaya.text.function.split_into_sentences(data['text'])
    #             if len(malaya.text.function.split_into_sentences(extractive)) <= (len(splitted) / 2):
    #                 accepted.append((extractive, data['text']))
    #         except Exception as e:
    #             print(e)

            count += 1

9622it [01:11, 134.35it/s]
7897it [00:42, 184.82it/s]
9956it [01:13, 135.53it/s]
8578it [00:59, 143.62it/s]
9970it [01:11, 139.64it/s]
9826it [01:06, 148.31it/s]
972it [00:06, 139.42it/s]
5342it [00:46, 114.30it/s]


In [27]:
with open('news-astroawani.json', 'w') as fopen:
    json.dump(accepted, fopen)

In [29]:
len(accepted)

178407